In [ ]:
import pandas as pd
import numpy as np

train_origin = pd.read_csv('../data/traintest_v2.2.csv')

In [ ]:
train_origin.shape

# Data Version 3.0

- 각 문제, tag, cat 등등 소요시간 잘못된 것 수정


In [ ]:
train = train_origin[['userID', 'Timestamp', 'assessmentItemID', 'test_cat', 'question_number', 'KnowledgeTag', 'week_num', 'testId']].copy()
train['post_Timestamp'] = train.groupby(['userID', 'testId'])['Timestamp'].shift(-1)
train = train.dropna(axis=0)
train['Timestamp'] = pd.to_datetime(train['Timestamp'])
train['post_Timestamp'] = pd.to_datetime(train['post_Timestamp'])
train['elapsed_time_now'] = (train['post_Timestamp'].astype(int) - train['Timestamp'].astype(int)) // 1e9
train['elapsed_time_now'] = train['elapsed_time_now'].astype(int)

train['elapsed_time_now'] = train['elapsed_time_now'].apply(lambda x: max(0, min(x, 600)))
train['elapsed_time_now_log'] = np.log(train['elapsed_time_now'] + 1)




In [ ]:
def et_mean_std(train, train_origin, column):
    col2mean = train.groupby(column)['elapsed_time_now_log'].mean().to_dict()
    col2std = train.groupby(column)['elapsed_time_now_log'].std().to_dict()

    train_origin[f'{column}_et_std'] = train_origin[column].map(col2mean)
    train_origin[f'{column}_et_mean'] = train_origin[column].map(col2std)
    return train_origin

et_mean_std(train, train_origin, 'assessmentItemID')
et_mean_std(train, train_origin, 'test_cat')
et_mean_std(train, train_origin, 'question_number')
et_mean_std(train, train_origin, 'KnowledgeTag')
et_mean_std(train, train_origin, 'week_num')
et_mean_std(train, train_origin, 'testId')


In [ ]:
train_origin['KnowledgeTag_et_mean'].fillna(np.mean(train_origin['KnowledgeTag_et_mean']), inplace=True)
train_origin['KnowledgeTag_et_std'].fillna(np.mean(train_origin['KnowledgeTag_et_std']), inplace=True)
train_origin['assessmentItemID_et_std'].fillna(np.mean(train_origin['assessmentItemID_et_std']), inplace=True)
train_origin['assessmentItemID_et_mean'].fillna(np.mean(train_origin['assessmentItemID_et_mean']), inplace=True)

In [ ]:
train_origin.to_csv('../data/traintest_v3.0.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

train_origin = pd.read_csv('../data/traintest_v3.0.csv')
train_origin.shape